**Project 2 Phase 2**<br>
Group 8
<br>
<table style="width:100%">
  <tr>
    <th style = "text-align: left">#</th>
    <th style = "text-align: left">Name</th>
    <th style = "text-align: left">Lastname</th>
    <th style = "text-align: left">Matr Number</th>
  </tr>
  <tr>
    <td style = "text-align: left">1</td>
    <td style = "text-align: left">Christian</td>
    <td style = "text-align: left">Peinthor</td>
    <td style = "text-align: left">11815592</td>
  </tr>
  <tr>
    <td style = "text-align: left">2</td>
    <td style = "text-align: left">Michael</td>
    <td style = "text-align: left">Weikl</td>
    <td style = "text-align: left">1154652</td>
  </tr>
  <tr>
    <td style = "text-align: left">3</td>
    <td style = "text-align: left">Uros</td>
    <td style = "text-align: left">Zivanovic</td>
    <td style = "text-align: left">12032271</td>
  </tr>
  <tr>
    <td style = "text-align: left">4</td>
    <td style = "text-align: left">Maximilian</td>
    <td style = "text-align: left">Hofmann</td>
    <td style = "text-align: left">12019987</td>
  </tr>
  
</table>
<br>

# Basic stuff

In [2]:
from typing import Union

import numpy as np
import numpy.linalg as lin
import scipy.optimize

In [3]:
class Problem():
    def f(self, x):
        raise NotImplementedError("f() is not implemented for this Problem")
    
    def grad_f(self, x):
        raise NotImplementedError("grad_f() is not implemented for this Problem")
            
    def hessian(self, x):
        raise NotImplementedError("hessian() is not implemented for this Problem")      
        

In [4]:
class PolynomialProblem(Problem):
        @property
        def actual_minima(self):
            raise NotImplementedError("actual_minima() is not implemented for this Problem")      


############################################################
#                Polynomial definitions
############################################################

class Rosenbrock(PolynomialProblem):
    def f(self, x):
        return 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2

    def grad_f(self, x):
        return np.array([-400*x[0]*(x[1] - x[0]**2) - 2*(1 - x[0]),
                         200*(x[1] - x[0]**2)])

    def hessian(self, x):
        return np.array([[-400*(x[1] - 3*x[0]**2) + 2, 
                          -400*x[0]],
                         [-400*x[0], 
                          200]])
    
    @property
    def actual_minima(self):
        return np.array([1,1])


#this is the function below rosenbrock defined in the project description
class Function2(PolynomialProblem):
    def f(self, x):
        return 150*(x[0] * x[1])**2 + (0.5 * x[0] + 2 * x[1] - 2)**2
    
    def grad_f(self, x):
        return np.array([x[0] * (300 * x[1]**2 + 0.5) + 2 * x[1] - 2, 300*x[0]**2 * x[1] + 2 * x[0] + 8 * x[1] - 8])

    def hessian(self, x):
        return np.array([[300 * x[1]**2 + 0.5, 600 * x[0]*x[1] + 2],[600 * x[0]*x[1] + 2, 300 * x[0]**2 + 8]])

    #self calculated minima
    @property
    def actual_minima(self):
        return np.array([[0,1],[4,0]])

# Implementations

## Simplex

In [5]:
def simplex_method(linear_problem: Union[np.ndarray, None], constraints: np.ndarray, constraint_rhs: np.ndarray):
    def swap(variables: list, exits_idx: int, enters_idx: int) -> None:
        """
        In-/excluding respective vectors into/from the basis.
        """
        tmp = variables[exits_idx]
        variables[exits_idx] = variables[enters_idx]
        variables[enters_idx] = tmp
    
    constraint_rhs = constraint_rhs.reshape((-1, 1))
    n_constraints = constraints.shape[0]  # Number of basis vectors
    n_orig_variables = constraints.shape[1]  # Number of original_variables in each constraint
    n_variables = constraints.shape[1]  # Number of variables in each constraint

    tableau = np.concatenate((constraints, np.eye(n_constraints)), axis=1)  # Extended matrix of the system by slack constraints
    x = np.concatenate((np.zeros([n_variables, 1]), constraint_rhs))  # Right parts of the constraints
    
    non_base_weigths = np.zeros((n_variables, 1)) if linear_problem is None else linear_problem
    base_weights = np.ones((n_constraints, 1)) if linear_problem is None else np.zeros((n_constraints, 1))
    weights = np.vstack((non_base_weigths, base_weights))
    
    variable_idx = list(range(len(weights)))  # Indices for all variables
    priv_x_B, x_B = None, None


    while True:
        basis_idx = variable_idx[n_variables:]
        basis = tableau.T[basis_idx].T
        basis_inv = lin.inv(basis)
        priv_x_B = x_B
        x_B = x[basis_idx]
        c_B = weights[basis_idx]
        lam = lin.inv(basis.T) @ c_B

        non_basis_idx = variable_idx[:n_variables]
        c_N = weights[non_basis_idx]
        n_T = tableau.T[non_basis_idx]
        s_N = c_N - (n_T @ lam)

        if np.all(s_N >= 0) or (x_B is not None and priv_x_B is not None and np.sum(priv_x_B - x_B) == 0):
            print("Optimal solution found!!!")
            solution = None if linear_problem is None else c_B.T @ basis_inv @ constraint_rhs
            return x[:n_orig_variables].flatten(), x[n_orig_variables:].flatten(), solution

        # since here must be at least one s_q < 0, we take the argmin to get the index of the smallest value.
        # q_idx = np.argmin(s_N)  # Fetch the idx in the non basis idx set
        # q = non_basis_idx[q_idx]  # Fetch the variable_idx
        # d = basis_inv @ tableau.T[q].T
        # d = np.reshape(d, (-1, 1))

        d = basis_inv @ tableau
        n_q = np.vstack((-d.T[:n_variables].T, np.identity(n_variables)))

        change = (weights.T @ n_q) / np.linalg.norm(n_q, axis=0)
        change[(s_N >= 0).T] = float('inf')
        q_idx = np.argmin(change)
        q = non_basis_idx[q_idx]

        if np.all(d <= 0):
            print("Problem is unbound!!!")
            return None, None, None
        d_q = np.reshape(d[:, q], (-1, 1))
        piv_weigths = x_B / d_q
        p_idx = np.argmin(piv_weigths)
        x_q = piv_weigths[p_idx]
        x[basis_idx] = x_B - (d_q * x_q)
        x[q] = x_q  # set entering variable vaLue

        # In-/excluding respective vectors into/from the basis:
        swap(variables=variable_idx, exits_idx=p_idx + n_variables, enters_idx=q_idx)

## Active Set

In [6]:
def compute_learning_rate(A, p, x, w, b):
    numerator = b - (A @ x)
    denominator = A @ p
    possible_alphas = numerator / denominator

    idx_gtz = np.where(denominator > 0)[0].tolist()

    if len(idx_gtz) == 0:
        return 1, None
    else:
        blocking = []
        for idx in idx_gtz:
            if idx not in w:
                blocking.append(idx)

        if len(blocking) == 0:
            return 1, None
        else:
            possible_alphas = possible_alphas[blocking]
            min_alpha = np.argmin(possible_alphas)
            if len(min_alpha.shape) != 0:
                min_alpha = min_alpha[0]

            if possible_alphas[min_alpha] < 1:
                return possible_alphas[min_alpha], min_alpha
            else:
                return 1, None


def compute_descent_direction_and_lagrangien(G, x, c, A, w):
    working_A = A[w]
    size_of_zeros = working_A.shape[0]

    problem = np.concatenate((
        np.concatenate((G, working_A.T), axis=1),
        np.concatenate((working_A, np.zeros((size_of_zeros, size_of_zeros))), axis=1)),
        axis=0)
    g_k = (G @ x) + c
    rhs = np.concatenate((-g_k, np.zeros(size_of_zeros)), axis=0)

    result = np.linalg.solve(problem, rhs.T)

    descent_direction = result[0:len(G)]
    lagrangiens = result[len(G):]

    return descent_direction, lagrangiens


def active_set_method(G, c, A, b, x0=None, steps=1000):
    if x0 is None:
        x, _, _ = simplex_method(None, A, b)
        print(f"No starting point given: Found one feasible point at '{x}'")
    else:
        x = np.copy(x0)
        
    w: list = np.where(A @ x == b)[0].tolist()

    for k in range(steps + 1):
        p, lambdas = compute_descent_direction_and_lagrangien(G=G, x=x, c=c, A=A, w=w)

        if np.all(np.abs(p) <= 1e-15):
            if np.all(lambdas >= 0):
                return x, k
            else:
                j = np.argmin(lambdas)
                w_j = w[j]
                while w_j in w:
                    w.remove(w_j)

        else:
            alpha, blocking_idx = compute_learning_rate(A, p, x, w, b)
            x = x + alpha * p

            # Are there blocking constraints
            # http://lendek.net/teaching/OPT/qp.pdf
            if alpha < 1:
                w.append(blocking_idx)

    return None, k

## SQP

In [7]:
### Implementation

## Derivative Approximations

In [8]:
def e_i(size, index):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr


# these return the function that computes the gradient, which can then in turn be called to compute the gradient
def approx_grad(f, e=1.1e-8):
    def grad_f(x):
        if x.size == 1:
            return (f(x + e) - f(x)) / e
        return np.array([(f(x + e * e_i(x.size, i)) - f(x)) / e for i in range(x.size)])

    return grad_f


def approx_hessian(f, e=1.1e-8):
    def hessian_f(x):
        if x.size == 1:
            return (f(x + 2 * e) - 2 * f(x + e) + f(x)) / e ** 2
        return np.array([[(f(x + e * e_i(x.size, i) + e * e_i(x.size, j)) - f(
            x + e * e_i(x.size, i)) - f(x + e * e_i(x.size, j)) + f(
            x)) / e ** 2 for j in range(x.size)] for i in range(x.size)])

    return hessian_f

# Testruns

## Simplex

In [9]:
prob = np.array([[-3], [-2]])
const = np.array([[1, 1], [2, 0.5]])
rhs = np.array([[5], [8]])

x_val, slack_val, sol = simplex_method(linear_problem=prob, constraints=const, constraint_rhs=rhs)
print(f"x = {x_val}")
print(f"slack = {slack_val}")
print(f"solution = {sol}")

Optimal solution found!!!
x = [3.66666667 1.33333333]
slack = [0. 0.]
solution = [[-13.66666667]]


## Generating and solving simplex problems

In [10]:
# working simplex as placeholder

def simplex(set_b, set_n, A, b, c, print_steps=False):

    A = A.T

    sB = np.zeros((len(set_b), 1))

    xN = np.zeros((len(set_n), 1))

    x = np.zeros((A.shape[0], 1))
    s = np.zeros((A.shape[0], 1))
    
    B = A[set_b].T
    N = A[set_n].T
    cB = c[set_b]
    cN = c[set_n]

    xB = np.linalg.inv(B) @ b

    while True:

        l = np.linalg.inv(B).T @ cB
        
        sN = cN - N.T @ l
        
        if print_steps:
            print(f'\n\nxB:\n{xB}\n\nsN:\n{sN}\n\nlambda:\n{l}\n\nf(x):\n{f(x[:2])}')

        if all(s >= 0 for s in sN):
            optimal_points = [p[0] for p in x]
            return optimal_points

        q = None
        min_change = np.inf
        d = None
        for i, k in enumerate(set_n):

            d_q = np.linalg.inv(B) @ np.reshape(A[k], (-1, 1))
            e = np.identity(c.shape[0] - d_q.shape[0])
            n_q = np.vstack((-d_q, np.reshape(e[i], (-1, 1))))

            change = (c.T @ n_q) / np.linalg.norm(n_q)

            if change < min_change and sN[i] < 0:
                min_change = change
                q = k
                d = d_q

        if all(d_i <= 0 for d_i in d):
            return False

        p = None
        minimum = np.inf
        for i in range(len(d)):
            if d[i] > 0:
                if xB[i] / d[i] < minimum:
                    minimum = xB[i] / d[i]
                    p = i

        x[q] = minimum
              
        xB = xB - d * x[q]  
        x[set_b] = xB[range(len(set_b))]

        xN[set_n.index(q)] = x[q]
        x[set_n] = xN[range(len(set_n))]

        index_n = set_b[p]

        set_b[p] = q
        set_n[set_n.index(q)] = index_n
        
        set_b_len = range(len(set_b))
        set_n_len = range(len(set_n))
        
        B[:, set_b_len] = A[set_b].T
        cB[set_b_len] = c[set_b]
        xB[set_b_len] = x[set_b]
        
        N[:, set_n_len] = A[set_n].T
        cN[set_n_len] = c[set_n]

In [11]:
def generate_problem(n_variables, n_constraints):
    A = np.random.randint(-10, 11, size=(n_constraints, n_variables))
    A = np.concatenate((A, np.identity(n_constraints)), axis=1)
    c = np.random.randint(-10, 11, size=(n_variables, 1))
    c = np.vstack((c, np.zeros((n_constraints, 1))))
    b = np.random.randint(0, 11, size=(n_constraints, 1))

    set_b = [i for i in range(A.shape[1] - n_constraints, A.shape[1])]
    set_n = [i for i in range(A.shape[1] - n_constraints)]

    return A, b, c, set_b, set_n

In [12]:
def generate_and_solve(n_problems, n_variables, n_constraints):
    for i in range(n_problems):

        A, b, c, set_b, set_n = generate_problem(n_variables, n_constraints)

        result = simplex(set_b, set_n, A, b, c)

        while result == False or np.sum(result) < 1e-15:
            A, b, c, set_b, set_n = generate_problem(n_variables, n_constraints)
            result = simplex(set_b, set_n, A, b, c)

        print(f'A:\n{A}\nb:\n{b}\nc:\n{c}\n')
        print(f'Minimum at: {result}\n\n')

In [13]:
generate_and_solve(n_problems=5, n_variables=10, n_constraints=5)

A:
[[-9. 10.  7.  1. -9.  5.  3.  7.  7.  9.  1.  0.  0.  0.  0.]
 [ 7. -4. -8.  1.  0.  9.  5.  9.  0. -2.  0.  1.  0.  0.  0.]
 [ 6. -4.  1. -7.  4.  9.  3.  6. -1.  1.  0.  0.  1.  0.  0.]
 [ 1.  3. -7. -7.  7.  6.  6. -8.  4. -6.  0.  0.  0.  1.  0.]
 [ 9. -5. 10. -1.  6.  4. -3. -6.  1.  6.  0.  0.  0.  0.  1.]]
b:
[[4]
 [8]
 [5]
 [6]
 [7]]
c:
[[  1.]
 [  8.]
 [-10.]
 [  9.]
 [ -3.]
 [ -5.]
 [ -4.]
 [ -1.]
 [ -4.]
 [  7.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]]

Minimum at: [0.4088669950738915, 0.0, 0.5945945945945945, 0.0, 0.5459459459459459, 0.0, 0.8459459459459455, 2.5505481120584643, 0.3162162162162166, 0.0, 0.017299375300335863, 8.527027027027026, 0.0023597273203984615, 0.0, 0.41379310344826387]


A:
[[  9.   0.  -8.   3. -10.  10.  -9.   2.   0.   4.   1.   0.   0.   0.
    0.]
 [ -2.   5.   4.  -9.   0.   5.   2.   4.   4. -10.   0.   1.   0.   0.
    0.]
 [  2.  -4.  -6.  -6.   3.  -1.   3.  -6.   0.   4.   0.   0.   1.   0.
    0.]
 [  7.   4.   9. -10.   0.   8.  -9.   

## Active Set

In [14]:
G = np.array([[-2, 0], [0, -2]])
c = np.array([6, 4])
A = np.array([[1, 1], [-1, 0], [0, -1]])
b = np.array([3, 0, 0])

# feasible_x, W = find_feasible_point(A, b, n_constraints=3, n_variables=2)
# print(feasible_x, W)

result, iterations = active_set_method(G, c, A, b)
print(f"Found solution at {result} after {iterations} iterations")

Optimal solution found!!!
No starting point given: Found one feasible point at '[0. 0.]'
Found solution at [0. 0.] after 0 iterations


## Generating and solving QP problems

In [81]:
# placeholder asm

def asm(G, W, A, x, c, b, iters=1000):

    for iteration in range(iters):

        A_k = A[W]
        g_k = G @ x + c

        # construct matrix of linear equations to solve for p_k and lambdas
        """
        form: [G, A.T] [p_k] = [-g_k]
              [A,  0 ] [ l ] = [  0 ]
        """
        top = np.hstack((G, A_k.T))
        bottom = np.hstack((A_k, np.zeros((A_k.shape[0], A_k.shape[0]))))
        big = np.vstack((top, bottom))

        # solve
        equation_vec = np.vstack((-g_k, np.zeros((A_k.shape[0], 1))))
        solution = np.linalg.lstsq(big, equation_vec)[0]

        p_k = solution[:G.shape[0]]

        if all(abs(p_i) < 1e-12 for p_i in p_k):

            l_vec = solution[G.shape[0]:]

            if all(l >= 0 for l in l_vec):
                return x

            else:
                j = np.argmin(l_vec)

                W.pop(j)
        else:

            a_k = 1
            blocking_constraints = []

            for i in range(A.shape[0]):

                if i not in W and A[i].T @ p_k > 0:
                    candidate = (b[i] - A[i] @ x) / (A[i] @ p_k)
                    if candidate <= a_k:
                        a_k = candidate
                        blocking_constraints.append(i)

            x = x + a_k * p_k

            if a_k < 1:
                W += [blocking_constraints.pop()]

In [77]:
def generate_and_solve_qp(m_rows=5, M=None, y=None, x0=None):
    
    if M is None:
        M = np.random.randint(-10, 10, size=(m_rows, 2*m_rows))
    
    if y is None:
        y = np.random.randint(-10, 10, size=(m_rows, 1))
    
    G = M.T @ M
    c = -(M.T @ y)
    b = np.ones((M.shape[1], 1), dtype=int)
    
    # for now random constraints
    
    A = np.random.randint(-20, 21, size=(b.shape[0], G.shape[0]))

    n_constraints = b.shape[0]
    n_variables = G.shape[0]
    
    A_simp = np.hstack((A, np.identity(n_constraints)))
    c_simp = np.ones((n_constraints, 1))
    c_simp = np.vstack((np.zeros((n_variables, 1)), c_simp))

    set_b = [i for i in range(A_simp.shape[1] - n_constraints, A_simp.shape[1])]
    set_n = [i for i in range(A_simp.shape[1] - n_constraints)]

    minimiser = simplex(set_b, set_n, A_simp, b, c_simp)

    x = np.reshape(minimiser[:n_variables], (-1, 1))
    z = minimiser[n_variables:]

    W = np.where(A @ x == b)[0].tolist()
    
    result = asm(G, W, A, x, c, b)

    return W, G, A, b, c, x, result

## 3 runs of QP Problem 1

In [79]:
W, G, A, b, c, x, result = generate_and_solve_qp(5)
iters = 1000

tries = 0
while result is None:
    W, G, A, b, c, x, result = generate_and_solve_qp(5)
    tries += 1

print(f'Feasible function found after {tries} attempts.')
print(f'W:\n{W}\nG:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
print(f'Minimum at:\n{result}\n\n')

x = np.reshape([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.')
    
x = np.reshape([1, 0, 1, 0, 1, 0, 1, 0, 1, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')

Feasible function found after 1 attempts.
W:
[3, 9, 5, 6, 8, 1]
G:
[[   9   10   15   -2  -10   -7  -18   20   -8  -14]
 [  10  234  129  187  -40  -41   -5   60   22  -53]
 [  15  129  252   27   74 -146  -20   20   62 -109]
 [  -2  187   27  225  -88   55   39   71   -1   -2]
 [ -10  -40   74  -88  100  -82  -20  -76   56  -28]
 [  -7  -41 -146   55  -82  128   14   24  -38   59]
 [ -18   -5  -20   39  -20   14  222   41  -56   70]
 [  20   60   20   71  -76   24   41  103  -54   -8]
 [  -8   22   62   -1   56  -38  -56  -54   63  -35]
 [ -14  -53 -109   -2  -28   59   70   -8  -35   66]]
A:
[[ -1  14 -10  -5   8 -14   3 -16  -5 -10]
 [-18  18  18  16  -8  -4   5  -5 -17   8]
 [ -1  -9   6  10  19  10  -1   5 -10  -7]
 [  2 -13   7   3 -17 -19  11  10  18 -11]
 [-12 -15  -2 -20   3  -3 -16 -12 -16   7]
 [-13  -8  10  11 -17   6  -7   8  13   3]
 [-10  -6 -18  10   7  15   6   9   0  -4]
 [ -4  17 -12  17 -12  13 -19  -9   7 -14]
 [-15  18  -7   1   0  -7  10 -19   7 -14]
 [ 17   3  1

## 3 runs of QP Problem 2

In [80]:
W, G, A, b, c, x, result = generate_and_solve_qp(5)
iters = 1000

tries = 0
while result is None:
    W, G, A, b, c, x, result = generate_and_solve_qp(5)
    tries += 1

print(f'Feasible function found after {tries} attempts.')
print(f'W:\n{W}\nG:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
print(f'Minimum at:\n{result}\n\n')

x = np.reshape([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')
    
x = np.reshape([1, 0, 1, 0, 1, 0, 1, 0, 1, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')

Feasible function found after 0 attempts.
W:
[8, 1, 4, 0, 9, 7]
G:
[[ 245 -168  -40  -61   35 -170  169  -16  -55  148]
 [-168  168   62   54 -110  204  -98   58    0 -106]
 [ -40   62   71   -7  -66   80   20   -4   -3  -42]
 [ -61   54   -7  191   56   16 -158  -34    2   79]
 [  35 -110  -66   56  202 -196  -43 -154   37   99]
 [-170  204   80   16 -196  280  -66  128  -22 -146]
 [ 169  -98   20 -158  -43  -66  235  -33  -64   36]
 [ -16   58   -4  -34 -154  128  -33  234   10  -88]
 [ -55    0   -3    2   37  -22  -64   10   66  -48]
 [ 148 -106  -42   79   99 -146   36  -88  -48  181]]
A:
[[ -4  12  -2   5 -20  -7   2  -1  12  15]
 [ -6   6 -20 -17  16  20 -19 -16  -2  20]
 [  9   8  15 -12 -12   6  19  -2  20 -18]
 [ 14  11   8   0  15 -17 -19 -14   9   1]
 [  5   1  12  18   5  11 -18  12  -2  -3]
 [ 19  -8 -12  -1  15  15  -6 -20 -16  18]
 [ 12  -7  -2  20   1 -14  19  15 -14 -12]
 [ -9  -2  -3  20 -13   3   5   8 -18 -19]
 [  5  -1 -16 -12  -7  14  -1  -2  16   7]
 [ -8  12   

## 4 runs of QP Problem 3

In [87]:
zero_matrix = np.zeros((2, 4), dtype=int)
M_hat = np.array([[1, 1, 0, 0], [0, 0, 1, 1]])

M_1 = np.hstack((M_hat, zero_matrix, zero_matrix, zero_matrix, zero_matrix))
M_2 = np.hstack((zero_matrix, M_hat, zero_matrix, zero_matrix, zero_matrix))
M_3 = np.hstack((zero_matrix, zero_matrix, M_hat, zero_matrix, zero_matrix))
M_4 = np.hstack((zero_matrix, zero_matrix, zero_matrix, M_hat, zero_matrix))
M_5 = np.hstack((zero_matrix, zero_matrix, zero_matrix, zero_matrix, M_hat))

M = np.vstack((M_1, M_2, M_3, M_4, M_5))
print((M).shape)

y = np.reshape([1, -2, 3, -4, 5, -5, 4, -3, 2, -1], (-1, 1))

W, G, A, b, c, x, result = generate_and_solve_qp(M=M, y=y)

print('\n\nRUN 1\n\n')
print(f'Feasible function found after {tries} attempts.')
print(f'W:\n{W}\nG:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
print(f'Minimum at:\n{result}\n\n')

print('\n\nRUN 2\n\n')
x = np.reshape([0]*20, (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')

print('\n\nRUN 1\n\n')
x = np.reshape([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0], (-1, 1))
W = []
print(f'W:\n{W}')
result = asm(G, W, A, x, c, b, iters)

print(f'G:\n{G}\nA:\n{A}\nb:\n{b}\nc:\n{c}\nx0:\n{x}\n')
if result is not None:
    print(f'Minimum at:\n{result}\n\n')
else:
    print(f'Minimum could not be found within {iters} iterations.\n\n')

(10, 20)
Feasible function found after 0 attempts.
W:
[12, 0, 10]
G:
[[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]]
A:
[[ -6  18  -6 -10 -11  -6   4 -15  -1  -7  -1  -5 -16 -19  18   6  

<ipython-input-81-d33afea8a4af>:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  solution = np.linalg.lstsq(big, equation_vec)[0]


## SQP